# Data Cleaning
Clean the raw data collected from the scrapper and prepare the data ready for exploratory data analysis

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("autotrader_prices.csv", parse_dates=["year"])

In [6]:
df.head()

,BHP,ULEZ,body,engine,fuel,link,mileage,name,owners,price,transmission,year
0,108.0,ULEZ,Hatchback,1.6L,Diesel,https://www.autotrader.co.uk/car-details/20200...,82850,Audi A3 1.6 TDI SE 5d 109 BHP Nav Prep - Bluet...,NaN,"£7,991",Manual,2015 (15 reg)
1,148.0,ULEZ,Hatchback,2.0L,Diesel,https://www.autotrader.co.uk/car-details/20201...,38627,"Audi A3 2.0 TDI SE 5d 148 BHP, £20 YEAR ROAD T...",NaN,"£10,980",Manual,2015 (65 reg)
2,108.0,ULEZ,Hatchback,1.6L,Diesel,https://www.autotrader.co.uk/car-details/20201...,84812,Audi A3 1.6 TDI 110 Sport 5dr Satellite Naviga...,NaN,"£8,350",Manual,2015 (15 reg)
3,182.0,ULEZ,Hatchback,2.0L,Diesel,https://www.autotrader.co.uk/car-details/20200...,74413,Audi A3 2.0 TDI Sport Sportback S Tronic quatt...,NaN,"£9,699",Automatic,2015 (15 reg)
4,108.0,ULEZ,Hatchback,1.6L,Diesel,https://www.autotrader.co.uk/car-details/20201...,59152,Audi A3 1.6 TDI SE Sportback 5dr,NaN,"£8,990",Manual,2015 (15 reg)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 12 columns):
BHP             469 non-null float64
ULEZ            617 non-null object
body            626 non-null object
engine          626 non-null object
fuel            625 non-null object
link            626 non-null object
mileage         626 non-null int64
name            626 non-null object
owners          34 non-null float64
price           626 non-null object
transmission    626 non-null object
year            626 non-null object
dtypes: float64(2), int64(1), object(9)
memory usage: 58.8+ KB


In [8]:
df.describe()

,BHP,mileage,owners
count,469.000000,626.000000,34.000000
mean,157.720682,27150.118211,2.264706
std,63.677983,18308.222440,0.567227
min,68.000000,10.000000,2.000000
25%,116.000000,14663.250000,2.000000
50%,148.000000,25153.500000,2.000000
75%,190.000000,35429.500000,2.000000
max,340.000000,124395.000000,4.000000


In [9]:
df.isna().sum()

BHP             157
ULEZ              9
body              0
engine            0
fuel              1
link              0
mileage           0
name              0
owners          592
price             0
transmission      0
year              0
dtype: int64

## Data cleaning

### Filling missing BHP
Tried to first replace with median value, however, need to use **a more sensible way of filling up this feature**

In [10]:
df_clean = df.copy()
df_clean["BHP"].fillna(df.BHP.median(), inplace=True)

### Filling ULEZ compliance

In [11]:
df_clean["ULEZ"].fillna("Non-ULEZ", inplace=True)

### Engine size parsing
Get rid of the Liter "L" and convert to float

In [12]:
df_clean["engine"] = df_clean["engine"].apply(lambda x: float(x[:-1]))

### Remove link

In [13]:
df_clean.drop(["link"], axis=1, inplace=True)

### Parsing name into multiple extra features

In [14]:
df_clean["make"] = df_clean["name"].apply(lambda x:x.split()[0])
df_clean["model"] = df_clean["name"].apply(lambda x:x.split()[1])

In [15]:
# diffentiate from "ECO", SE", "Sport", "M Sport", "Lux", Nav/Satellite", "Bluetooth",
# "Leather", "3-door", 
df_clean["se"] = df_clean["name"].apply(lambda x: 1 if "se" in x.lower() else 0)
df_clean["se"].value_counts()

1    384
0    242
Name: se, dtype: int64

In [16]:
# maybe too small/biased to be relevant
df_clean["eco"] = df_clean["name"].apply(lambda x: 1 if "eco" in x.lower() else 0)
df_clean["eco"].value_counts()

0    622
1      4
Name: eco, dtype: int64

In [17]:
df_clean["sport"] = df_clean["name"].apply(lambda x: 1 if "sport" in x.lower() else 0)
df_clean["sport"].value_counts()

1    380
0    246
Name: sport, dtype: int64

In [18]:
df_clean["m_sport"] = df_clean["name"].apply(lambda x: 1 if "m sport" in x.lower() else 0)
df_clean["m_sport"].value_counts()

0    443
1    183
Name: m_sport, dtype: int64

In [19]:
# maybe too small/biased to be relevant
df_clean["lux"] = df_clean["name"].apply(lambda x: 1 if "lux" in x.lower() else 0)
df_clean["lux"].value_counts()

0    620
1      6
Name: lux, dtype: int64

In [20]:
df_clean["sat_nav"] = df_clean["name"].apply(lambda x: 1 if "sat" in x.lower() or "nav" in x.lower() else 0)
df_clean["sat_nav"].value_counts()

0    522
1    104
Name: sat_nav, dtype: int64

In [21]:
df_clean["bluetooth"] = df_clean["name"].apply(lambda x: 1 if "bluetooth" in x.lower() else 0)
df_clean["bluetooth"].value_counts()

0    605
1     21
Name: bluetooth, dtype: int64

In [22]:
df_clean["leather"] = df_clean["name"].apply(lambda x: 1 if "leather" in x.lower() else 0)
df_clean["leather"].value_counts()

0    605
1     21
Name: leather, dtype: int64

In [23]:
df_clean["three_door"] = df_clean["name"].apply(lambda x: 1 if "3-door" in x.lower() else 0)
df_clean["three_door"].value_counts()

0    621
1      5
Name: three_door, dtype: int64

### Filling missing owners
Fill missing values with median value

In [24]:
df_clean["owners"].fillna(df.owners.median(), inplace=True)

### Parsing years

In [25]:
df_clean["year_reg"] = df_clean["year"].apply(lambda x: x.split("(")[1].replace(")",""))
df_clean["year"] = df_clean["year"].apply(lambda x: int(x.split("(")[0]))

### Parsing price

In [26]:
df_clean["price"] = df_clean["price"].apply(lambda x: int(x.replace("£","").replace(",","")))

### Parsing fuel

In [27]:
df_fuel = df_clean[df_clean.fuel.isna()]
df_fuel.make

618    Fiat
Name: make, dtype: object

In [28]:
df_fuel = df_clean[df_clean.make == "Fiat"]
df_fuel.fuel.value_counts()

Petrol    65
Name: fuel, dtype: int64

In [29]:
df_clean["fuel"].fillna("Petrol", inplace=True)

### Check and save

In [30]:
df_clean.head()

,BHP,ULEZ,body,engine,fuel,mileage,name,owners,price,transmission,...,se,eco,sport,m_sport,lux,sat_nav,bluetooth,leather,three_door,year_reg
0,108.0,ULEZ,Hatchback,1.6,Diesel,82850,Audi A3 1.6 TDI SE 5d 109 BHP Nav Prep - Bluet...,2.0,7991,Manual,...,1,0,0,0,0,1,1,0,0,15 reg
1,148.0,ULEZ,Hatchback,2.0,Diesel,38627,"Audi A3 2.0 TDI SE 5d 148 BHP, £20 YEAR ROAD T...",2.0,10980,Manual,...,1,0,0,0,0,1,0,0,0,65 reg
2,108.0,ULEZ,Hatchback,1.6,Diesel,84812,Audi A3 1.6 TDI 110 Sport 5dr Satellite Naviga...,2.0,8350,Manual,...,0,0,1,0,0,1,0,0,0,15 reg
3,182.0,ULEZ,Hatchback,2.0,Diesel,74413,Audi A3 2.0 TDI Sport Sportback S Tronic quatt...,2.0,9699,Automatic,...,0,0,1,0,0,0,0,0,0,15 reg
4,108.0,ULEZ,Hatchback,1.6,Diesel,59152,Audi A3 1.6 TDI SE Sportback 5dr,2.0,8990,Manual,...,1,0,1,0,0,0,0,0,0,15 reg


In [32]:
df_clean.to_csv("autotrader_prices_cleaned.csv", index=False)